<a href="https://colab.research.google.com/github/anirbanghoshsbi/.github.io/blob/master/experimental/ML_model(Resampling).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [65]:
#!pip install yfinance
!pip install pandas-ta==0.2.45b

In [66]:
import yfinance as yf
import numpy as np
import pandas as pd
import pandas_ta as ta

In [67]:
nifty = yf.download('^nsei',start='2014-01-01',end='2020-12-31')

[*********************100%***********************]  1 of 1 completed


In [68]:
nifty.tail(2)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-12-29,13910.349609,13967.599609,13859.900391,13932.599609,13932.599609,439600
2020-12-30,13980.900391,13997.000000,13864.950195,13981.950195,13981.950195,380700


In [69]:
nifty['close_2%']=np.abs(nifty['Adj Close'].pct_change()*100).fillna(0)

In [70]:
test=pd.DataFrame(data=nifty['close_2%'].values,columns=['sum'])

In [71]:
nifty['cumsum']=0
nifty=nifty.reset_index()

In [72]:
for index , val in enumerate(nifty['close_2%']):
  
  if index==0:
    nifty.loc[index,'cumsum']=val
  else:
    if nifty['cumsum'].iloc[index-1]>2:
      nifty.loc[index,'cumsum']=val
    else:
      nifty.loc[index, 'cumsum']=val+nifty['cumsum'].iloc[index-1]    

In [73]:
#test_f=pd.DataFrame({'Date':nifty.index, 'sum':test['cumsum'].values})
#pd.DataFrame({'Date':df.index,'spread':(df['Adj Close'].values - mean)})

In [74]:
nifty['diff']=nifty['close_2%']-nifty['cumsum']

In [75]:
nifty.head(10)

,Date,Open,High,Low,Close,Adj Close,Volume,close_2%,cumsum,diff
0,2014-01-02,6301.250000,6358.299805,6211.299805,6221.149902,6221.149902,158100,0.000000,0.000000,0.000000
1,2014-01-03,6194.549805,6221.700195,6171.250000,6211.149902,6211.149902,139000,0.160742,0.160742,0.000000
2,2014-01-06,6220.850098,6224.700195,6170.250000,6191.450195,6191.450195,118300,0.317167,0.477909,-0.160742
3,2014-01-07,6203.899902,6221.500000,6144.750000,6162.250000,6162.250000,138600,0.471621,0.949530,-0.477909
4,2014-01-08,6178.049805,6192.100098,6160.350098,6174.600098,6174.600098,146900,0.200415,1.149945,-0.949530
5,2014-01-09,6181.700195,6188.049805,6148.250000,6168.350098,6168.350098,150100,0.101221,1.251167,-1.149945
6,2014-01-10,6178.850098,6239.100098,6139.600098,6171.450195,6171.450195,159900,0.050258,1.301425,-1.251167
7,2014-01-13,6189.549805,6288.200195,6189.549805,6272.750000,6272.750000,135000,1.641426,2.942851,-1.301425
8,2014-01-14,6260.250000,6280.350098,6234.149902,6241.850098,6241.850098,110200,0.492605,0.492605,0.000000
9,2014-01-15,6265.950195,6325.200195,6265.299805,6320.899902,6320.899902,145900,1.266448,1.759054,-0.492605


In [76]:
final_data=nifty.iloc[nifty[nifty['diff']==0].index-1][1:]

In [77]:
days_out=1

In [78]:
final_data['Signal_Flag'] = final_data['Close'].transform(lambda x : np.sign(x.diff(days_out)))

In [79]:
final_data=final_data.dropna()

In [80]:
final_data

,Date,Open,High,Low,Close,Adj Close,Volume,close_2%,cumsum,diff,Signal_Flag
7,2014-01-13,6189.549805,6288.200195,6189.549805,6272.750000,6272.750000,135000,1.641426,2.942851,-1.301425,1.0
11,2014-01-17,6306.250000,6327.100098,6246.350098,6261.649902,6261.649902,167700,0.906012,2.696707,-1.790695,-1.0
16,2014-01-24,6301.649902,6331.450195,6263.899902,6266.750000,6266.750000,160300,1.243370,2.579192,-1.335822,1.0
17,2014-01-27,6186.299805,6188.549805,6130.250000,6135.850098,6135.850098,190400,2.088800,2.088800,0.000000,-1.0
22,2014-02-03,6058.799805,6074.850098,5994.450195,6001.799805,6001.799805,134900,1.440187,2.715307,-1.275120,-1.0
...,...,...,...,...,...,...,...,...,...,...,...
1695,2020-12-08,13393.849609,13435.450195,13311.049805,13392.950195,13392.950195,543200,0.278533,2.150226,-1.871693,1.0
1700,2020-12-15,13547.200195,13589.650391,13447.049805,13567.849609,13567.849609,497600,0.071538,2.055172,-1.983634,1.0
1704,2020-12-21,13741.900391,13777.500000,13131.450195,13328.400391,13328.400391,687300,3.140495,4.555337,-1.414842,-1.0
1706,2020-12-23,13473.500000,13619.450195,13432.200195,13601.099609,13601.099609,458700,1.001016,2.035644,-1.034628,1.0


In [81]:
final_data.Signal_Flag.value_counts()

 1.0    252
-1.0    207
 0.0      1
Name: Signal_Flag, dtype: int64

# Features Creation 

In [87]:
final_data=final_data.reset_index(drop=True)

In [ ]:
final_data.ta.macd(append=True)
final_data.ta.vortex(append=True)
final_data.ta.aroon(append=True)
final_data.ta.mom(append=True)
final_data.ta.stdev(append=True)
final_data.ta.linreg(append=True)
final_data['spread']=final_data['LR_14']-final_data['close']
print()

In [84]:
final_data=final_data.dropna()

In [88]:
final_data

,date,open,high,low,close,adj_close,volume,close_2%,cumsum,diff,Signal_Flag,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,VTXP_14,VTXM_14,AROOND_14,AROONU_14,AROONOSC_14,MOM_10,STDEV_30,LR_14,spread
0,2014-07-23,7794.899902,7809.200195,7752.899902,7795.750000,7795.750000,122100,0.359171,2.230077,-1.870906,1.0,398.406783,-45.989851,444.396634,1.077150,0.702451,0.000000,100.000000,100.000000,379.899902,577.805790,7707.310847,-88.439153
1,2014-07-30,7746.200195,7798.700195,7707.600098,7791.399902,7791.399902,178100,0.551056,2.046739,-1.495683,-1.0,391.391982,-42.403721,433.795703,1.002207,0.847794,0.000000,92.857143,92.857143,208.000000,565.491559,7732.783771,-58.616132
2,2014-08-01,7662.500000,7716.700195,7593.899902,7602.600098,7602.600098,181100,1.537302,2.437013,-0.899711,-1.0,366.374795,-53.936727,420.311522,0.882128,0.810346,7.142857,85.714286,78.571429,60.500000,538.956737,7718.910378,116.310281
3,2014-08-06,7726.149902,7740.950195,7658.950195,7672.049805,7672.049805,116400,0.961718,2.846419,-1.884700,1.0,348.139371,-57.737721,405.877092,0.937209,0.766366,0.000000,78.571429,78.571429,113.849609,516.747318,7700.772722,28.722917
4,2014-08-11,7619.850098,7635.549805,7598.600098,7625.950195,7625.950195,107000,0.758407,2.110595,-1.352188,-1.0,326.207496,-63.735677,389.943172,0.903786,0.870860,0.000000,71.428571,71.428571,45.750000,488.583933,7681.349903,55.399708
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422,2020-12-08,13393.849609,13435.450195,13311.049805,13392.950195,13392.950195,543200,0.278533,2.150226,-1.871693,1.0,597.680466,104.251233,493.429233,1.089480,0.623329,0.000000,100.000000,100.000000,1455.299805,699.282520,13066.840543,-326.109652
423,2020-12-15,13547.200195,13589.650391,13447.049805,13567.849609,13567.849609,497600,0.071538,2.055172,-1.983634,1.0,635.378647,113.559531,521.819116,1.093801,0.603470,35.714286,100.000000,64.285714,1678.449219,754.793203,13276.337612,-291.511998
424,2020-12-21,13741.900391,13777.500000,13131.450195,13328.400391,13328.400391,687300,3.140495,4.555337,-1.414842,-1.0,638.572114,93.402398,545.169716,0.994754,0.627792,28.571429,100.000000,71.428571,1686.000000,790.466148,13412.707933,84.307542
425,2020-12-23,13473.500000,13619.450195,13432.200195,13601.099609,13601.099609,458700,1.001016,2.035644,-1.034628,1.0,655.550727,88.304809,567.245918,1.071195,0.653363,21.428571,92.857143,71.428571,1692.599609,839.150636,13551.522182,-49.577427


# and Modelling Part remaining